# Bedrock Converse Multi Agent Implementation

## Introduction

Based on the user input, a **Router implemented using Bedrock tool use invokes an Agent** from the pool of available agents defined as tools. Each agent will decide and invoke the required **tool(s)** to get the response back user.

This notebook has some borrowed code from [this](https://community.aws/content/2hW7srTWRb5idHjY4I8WP5fQFRf/build-a-tool-use-based-agent-loop-with-amazon-bedrock?lang=en) post

In [1]:
import boto3

In [2]:
class ToolError(Exception):
    pass

## Define tools for the task agents

In [3]:
# Tools for firewallagent1 Agent

def online_firewalls(arg):
    return {"firewall1": "A", "firewall2": "B"}

def offline_firewalls(arg):
    return [{"firewallA": 1}, {"firewallB": 2}]

firewallagent1_agent_tools = [
    {
        "toolSpec": {
            "name": "online_firewalls",
            "description": "This tool provides json of online firewalls",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The entire user query"
                        }
                    },
                    "required": ["query"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "offline_firewalls",
            "description": "This tool provides json of offline firewalls",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The entire user query"
                        }
                    },
                    "required": ["query"]
                }
            }
        }
    }
]

In [4]:
# Tools for firewallagent2 Agent

def all_firewalls(arg):
    all_firewalls = {"firewall1": "A", "firewall2": "B", "firewall3": "C", "firewall4": "D", "firewall5": "E", "firewall6": "F"}
    return all_firewalls

firewallagent2_agent_tools = [
    {
        "toolSpec": {
            "name": "all_firewalls",
            "description": "This tool provides json of all firewalls",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The entire user query"
                        }
                    },
                    "required": ["query"]
                }
            }
        }
    }
]

## Define helper function to call Bedrock

In [5]:
def call_bedrock(system_message, message_list, tool_list):
    session = boto3.Session()

    bedrock = session.client(service_name='bedrock-runtime')
    
    response = bedrock.converse(
        modelId="anthropic.claude-3-sonnet-20240229-v1:0",
        messages=message_list,
        system=system_message,
        inferenceConfig={
            "maxTokens": 2000,
            "temperature": 0
        },
        toolConfig={ "tools": tool_list }
    )
    
    return response


## Define helper function to invoke a given tool

In [6]:
def get_tool_result(tool_use_block):
    
    tool_name = tool_use_block['name']
    tool_input = tool_use_block['input']['query']
    
    print(f"Using tool `{tool_name}` for query `{tool_input}`")
    func = globals()[tool_name]
    try:
        return func(tool_input)
    except Exception as e:
        raise ToolError(f"Something went wrong: {e}")


## Define function to handle the raw responses from Bedrock

In [7]:
def handle_model_response(response):
    
    response_content_blocks = response['content']
    follow_up_content_blocks = []
    
    for content_block in response_content_blocks:
        if 'toolUse' in content_block:
            tool_use_block = content_block['toolUse']
            
            try:
                tool_result_value = get_tool_result(tool_use_block)
                
                if tool_result_value is not None:
                    follow_up_content_blocks.append({
                        "toolResult": {
                            "toolUseId": tool_use_block['toolUseId'],
                            "content": [
                                { "json": { "result": tool_result_value } }
                            ]
                        }
                    })
                
            except ToolError as e:
                follow_up_content_blocks.append({ 
                    "toolResult": {
                        "toolUseId": tool_use_block['toolUseId'],
                        "content": [  { "text": repr(e) } ],
                        "status": "error"
                    }
                })
        
    
    if len(follow_up_content_blocks) > 0:
        
        follow_up_message = {
            "role": "user",
            "content": follow_up_content_blocks,
        }
        
        return follow_up_message
    else:
        return None


## Define function that implements the Agent Loop till final response is received

In [8]:
def run_agent(system_prompt, input_prompt, tool_list):
    # MAX_LOOPS = 6
    # loop_count = 0
    continue_loop = True
    output = ""
    
    system_message = [
        {
            "text": system_prompt
        }
    ]
    
    message_list = [
        {
            "role": "user",
            "content": [ { "text": input_prompt } ]
        }
    ]
    
    while continue_loop:
        response = call_bedrock(system_message, message_list, tool_list)

        response_message = response['output']['message']
        message_list.append(response_message)

        # loop_count = loop_count + 1

        # if loop_count >= MAX_LOOPS:
        #     print(f"Hit loop limit: {loop_count}")
        #     break

        follow_up_message = handle_model_response(response_message)

        if follow_up_message is None:
            # No remaining work to do, return final response to user
            continue_loop = False
            if response['stopReason'] == "end_turn":
                agent_output = response_message['content'][0]['text']
        else:
            message_list.append(follow_up_message)
            
    return message_list, agent_output


## Define task agent executors

In [9]:
# firewallagent1 Agent Executor

def firewallagent1_agent_executor(firewallagent1_agent_input_prompt):
    
    firewallagent1_agent_system_prompt = """
    You are an agent to list down Online and Offline Firewalls
    """

    firewallagent1_agent_workflow, firewallagent1_agent_output = run_agent(firewallagent1_agent_system_prompt, firewallagent1_agent_input_prompt, firewallagent1_agent_tools)
    
    # returning just the output for now
    # can choose to return the workflow `firewallagent1_agent_workflow` as well which includes details of the different tools called when this agent is called
    
    return firewallagent1_agent_output

In [10]:
# firewallagent2 Agent Executor

def firewallagent2_agent_executor(firewallagent2_agent_input_prompt):
    
    firewallagent2_agent_system_prompt = """
    You are an agent to list all available Firewalls
    """

    firewallagent2_agent_workflow, firewallagent2_agent_output = run_agent(firewallagent2_agent_system_prompt, firewallagent2_agent_input_prompt, firewallagent2_agent_tools)
    
    # returning just the output for now
    # can choose to return the workflow `firewallagent2_agent_workflow` as well which includes details of the different tools called when this agent is called
    return firewallagent2_agent_output

## Execute task agents standalone

In [11]:
firewallagent1_agent_executor("Which firewalls are online")

Using tool `online_firewalls` for query `Which firewalls are online`


'The online firewalls according to the tool output are:\n- firewall1: A\n- firewall2: B'

In [12]:
firewallagent1_agent_executor("Which firewalls are offline")

Using tool `offline_firewalls` for query `Which firewalls are offline`


"The offline firewalls based on the tool's output are:\n- firewallA\n- firewallB"

In [13]:
firewallagent2_agent_executor("list all my firewalls")

Using tool `all_firewalls` for query `list all my firewalls`


'The tool all_firewalls returned a JSON object listing all your firewalls. The firewalls are:\n\nfirewall1: A\nfirewall2: B  \nfirewall3: C\nfirewall4: D\nfirewall5: E\nfirewall6: F'

## Define Task agents as tools for router agents

In [14]:
# Tools for Router Agent

router_agent_tools = [
    {
        "toolSpec": {
            "name": "firewallagent1_agent_executor",
            "description": "this is firewallagent1 Agent that returns list of firewalls that are online and offline",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The entire input from the user"
                        }
                    },
                    "required": ["query"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "firewallagent2_agent_executor",
            "description": "This is firewallagent2 Agent that returns inventory of all firewalls",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The entire input from the user"
                        }
                    },
                    "required": ["query"]
                }
            }
        }
    }
]

## Define router agent executor

In [15]:
# Router Agent Executor

def router_agent_executor(router_input_prompt):
    
    router_agent_system_prompt = """
    You are an agent to help with firewall queries, you use the tools at your disposal to fulfill user queries. 
    After you have the appropriate tool(s) and there is a final response, respond back with this output to the user
    """

    router_agent_workflow, router_agent_output = run_agent(router_agent_system_prompt, router_input_prompt, router_agent_tools)
    
    return router_agent_workflow, router_agent_output

## Run task agent executor

In [16]:
router_agent_workflow, router_agent_output = router_agent_executor("list all my firewalls")

Using tool `firewallagent2_agent_executor` for query `list all my firewalls`
Using tool `all_firewalls` for query `list all my firewalls`


In [17]:
router_agent_workflow

[{'role': 'user', 'content': [{'text': 'list all my firewalls'}]},
 {'role': 'assistant',
  'content': [{'toolUse': {'toolUseId': 'tooluse_sd5vkwOwSH2ehQh99yIigg',
     'name': 'firewallagent2_agent_executor',
     'input': {'query': 'list all my firewalls'}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'tooluse_sd5vkwOwSH2ehQh99yIigg',
     'content': [{'json': {'result': 'The tool all_firewalls returned a JSON object listing all your firewalls. The firewalls are:\n\nfirewall1: A\nfirewall2: B  \nfirewall3: C\nfirewall4: D\nfirewall5: E\nfirewall6: F'}}]}}]},
 {'role': 'assistant',
  'content': [{'text': 'The tool firewallagent2_agent_executor was used to list all your firewalls. The firewalls returned are: firewall1, firewall2, firewall3, firewall4, firewall5, firewall6.'}]}]

In [18]:
router_agent_output

'The tool firewallagent2_agent_executor was used to list all your firewalls. The firewalls returned are: firewall1, firewall2, firewall3, firewall4, firewall5, firewall6.'